Global Pruning of The Blood-brain barrier penetration (BBBP)
--------------------------------------------------------------

### Libraries

In [1]:
import os
import os.path as osp
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import time
import psutil
import itertools
import tracemalloc
import gc
import torch
import torch.nn as nn
from torch.optim import Adam
import statistics as stat


#********************************************************#
'''
load_dataset contain lots of functions for loading several datasets and 
also there is a function as name get_ dataloader for generating a
dictionary of training, validation, and testing dataLoader.
'''
from load_dataset import get_dataset, get_dataloader

#********************************************************#
'''
As we need several arguments for training process, we store all argument in configure file. 
For using this file, you need the library'Typed Argument Parser (Tap). So you need 'pip install typed-argument-parser'. 
'''
from Configures import data_args, train_args, model_args

'''
Libraries in pytorch for pruning
'''
from torch.nn.utils.prune import global_unstructured, L1Unstructured
import torch.nn.utils.prune as prune

### Sparsity
- The sparsity is the parameter that is determines the rate of pruning across the layer. It is a value in range(0,0.1,1). This parameter is fixed for this notebook and change for remaining experiment. We determine it before training process. Here is all values of sparsities. 

In [2]:
0.1, 0.2, 0.3, 0.4,  0.5, 0.6 , 0.7,  0.8, 0.9
;

''

### Pruning function and fuction for loading pruned model

In [3]:
## New version

import torch
import torch.nn as nn
from torch.nn.utils.prune import global_unstructured, L1Unstructured, remove

def global_pruning_torch(model, parameters_to_prune, sparsity, path):
    # Pruning the model layers globally using PyTorch library
    global_unstructured(parameters_to_prune, pruning_method=L1Unstructured, amount=sparsity)
    
    # Removing pruning reparameterization to save only the final weights
    for module, name in parameters_to_prune:
        remove(module, name)
   
    
    state = state_sparse_model(model, eval_acc=None)
    torch.save(state, path)
    pruned_model_size = os.path.getsize(path)
   
    return model

def state_sparse_model(model, eval_acc=None, epoch=None):
    state_dict = model.state_dict()
    compressed_state = {}
    
    for k, v in state_dict.items():
        if torch.is_tensor(v):
            mask = v != 0
            if mask.any():  # Only compress if there are non-zeros
                compressed_state[k] = {
                    'shape': v.shape,
                    'values': v[mask]  # Store only non-zero values
                }
            else:
                compressed_state[k] = v  # Keep original if all zeros
        else:
            compressed_state[k] = v
    
    return {'net': compressed_state, 'epoch': epoch, 'acc': eval_acc}

def load_sparse_model(state_path, original_model):
    """
    Loads a model saved in the custom compressed format (non-zero values only).
    Reconstructs dense tensors before loading into the model.
    """
    # Load the compressed state_dict
    compressed_state = torch.load(state_path)
    compressed_weights = compressed_state['net']
    
    # Initialize a new state_dict for the original model
    new_state_dict = original_model.state_dict()
    
    for k, v in compressed_weights.items():
        if isinstance(v, dict) and 'shape' in v and 'values' in v:
            # Reconstruct dense tensor from compressed format
            dense_tensor = torch.zeros(v['shape'], dtype=v['values'].dtype)
            mask = (dense_tensor != 0)  # All False initially
            # We need to know the positions of non-zero values (if available)
            # If indices were saved, use them; otherwise, assume sequential filling (simpler but may not match original positions)
            if 'indices' in v:
                # If you saved indices (advanced version)
                dense_tensor[v['indices']] = v['values']
            else:
                # If only values were saved (simpler version)
                # Flatten and fill non-zeros sequentially (may not match original positions)
                flat_tensor = dense_tensor.view(-1)
                flat_tensor[:len(v['values'])] = v['values']
                dense_tensor = flat_tensor.reshape(v['shape'])
            
            new_state_dict[k] = dense_tensor
        else:
            # If it's a normal tensor (e.g., biases, batch norm stats)
            new_state_dict[k] = v
    
    # Load the reconstructed state_dict
    original_model.load_state_dict(new_state_dict, strict=False)
    return original_model
    
def load_and_evaluate_pruned_model(model_args, model_path):
    """
    This function loads the pruned model from disk and evaluates it.
    """
    # Instantiate the model
    model = GnnNets(input_dim, output_dim, model_args)

    # Load the pruned model from disk and change arcithechture to compute accuracy
    sparse_model= load_sparse_model(model_path, model)
    
    print("Pruned model loaded.")
    
    return sparse_model


### Functions for Mmeasuring criterias

In [4]:
def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements

# Function to get CPU usage
def get_cpu_usage():
    return psutil.cpu_percent(interval=1)



# Function to approximate power consumption (Assume some average power usage per CPU percentage point)
def estimate_power_usage(cpu_usage):
    base_power_usage = 10  # Assumed base power usage in watts
    power_per_percent = 0.5  # Assumed additional watts per CPU usage percent
    return base_power_usage + (power_per_percent * cpu_usage)

# The model size based on the number of parameters
def get_model_size_param(model: nn.Module, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width


def get_model_sparsity(model: nn.Module) -> float:
    ''' 
    The input is layers of pruned model and the output is the sparsity after pruning.
    '''
    Sparsity=dict()
    global_zero=0
    global_nzero=0
    layyers=[]
    spars=[]
    for name, param in model.named_parameters(): 
        if 'weight' in name:
                    zero=float(torch.sum(param == 0))
                    nzero=float(param.nelement())
                    sparsity=  float(zero)/ float(nzero)
                    print( f'Sparsity in {name}: {sparsity:.3f}' )
                    layyers.append(name)
                    spars.append(sparsity)
                    global_zero +=zero
                    global_nzero +=nzero



    Sparsity={key: value for key, value in zip(layyers,spars)}
    global_sparsity= float(global_zero) /float(global_nzero)
    Sparsity.update({'Global sparsity':  global_sparsity})
    print("Global sparsity: {:.3f}".format(global_sparsity))
    return   Sparsity   

### start loading data

In [5]:
print(data_args.dataset_name)
print(data_args.dataset_dir)


bbbp
/datasets


In [6]:
dataset = get_dataset(data_args.dataset_dir, data_args.dataset_name)
input_dim = dataset.num_node_features
output_dim = int(dataset.num_classes)


print(input_dim)
print(output_dim)

9
2


C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


### Data Analysis

In [7]:
avg_nodes = 0.0
avg_edge_index = 0.0
for i in range(len(dataset)):
    avg_nodes += dataset[i].x.shape[0]
    avg_edge_index += dataset[i].edge_index.shape[1]
avg_nodes /= len(dataset)
avg_edge_index /= len(dataset)
print(f"graphs {len(dataset)}, avg_nodes{avg_nodes :.4f}, avg_edge_index_{avg_edge_index/2 :.4f}")

best_acc = 0.0
data_size = len(dataset)
print(f'The total num of dataset is {data_size}')



graphs 2050, avg_nodes23.9356, avg_edge_index_25.8151
The total num of dataset is 2050


### Preprocessing and cleaning dataset

In [8]:
#cleaned_dataset = [graph for graph in dataset if graph.edge_index.numpy()!=[]]
cleaned_dataset = [graph for graph in dataset if graph.edge_index.numpy().size> 0]
cleaned_dataset_len=len(cleaned_dataset)
print(f'The number of graphs after cleaning dataset is: {cleaned_dataset_len}')

The number of graphs after cleaning dataset is: 2039


In [9]:
dataloader=get_dataloader(cleaned_dataset, batch_size=train_args.batch_size, random_split_flag=True, data_split_ratio=[0.8, 0.1, 0.1], seed=2)

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


### Model for training

In [10]:
from GCN import GCNNet

def get_model(input_dim, output_dim, model_args):
    if model_args.model_name.lower() == 'gcn':
        return GCNNet(input_dim, output_dim, model_args)
    elif model_args.model_name.lower() == 'gat':
        return GATNet(input_dim, output_dim, model_args)
    elif model_args.model_name.lower() == 'gin':
        return GINNet(input_dim, output_dim, model_args)
    else:
        raise NotImplementedError
        


class GnnBase(nn.Module):
    def __init__(self):
        super(GnnBase, self).__init__()

    def forward(self, data):
        data = data.to(self.device)
        logits, prob, emb = self.model(data)
        return logits, prob, emb

    def update_state_dict(self, state_dict):
        original_state_dict = self.state_dict()
        loaded_state_dict = dict()
        for k, v in state_dict.items():
            if k in original_state_dict.keys():
                loaded_state_dict[k] = v
        self.load_state_dict(loaded_state_dict)

    def to_device(self):
        self.to(self.device)

    def save_state_dict(self):
        pass


class GnnNets(GnnBase):
    def __init__(self, input_dim, output_dim, model_args):
        super(GnnNets, self).__init__()
        self.model = get_model(input_dim, output_dim, model_args)
        self.device = model_args.device

    def forward(self, data):
        data = data.to(self.device)
        logits, prob, emb = self.model(data)
        return logits, prob, emb



In [11]:
model = GnnNets(input_dim, output_dim, model_args)
model.to_device()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=train_args.learning_rate, weight_decay=train_args.weight_decay)

### Function for training

In [12]:
def train(callbacks = None):
    logits, probs, _ = model(batch)
    loss = criterion(logits, batch.y)

    # optimization
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=2.0)
    optimizer.step()
    
    ## record
    _, prediction = torch.max(logits, -1)
    loss_list.append(loss.item())
    acc.append(prediction.eq(batch.y).cpu().numpy())
    
    
    # If callbacks is not None, it is the pruning function.
    # Infact, for fine-tuning of pruned model, after training in each epoch, model should be pruned. 
    if callbacks is not None:
        for callback in callbacks:
                callback()
  
    return acc


def evaluate_GC(eval_dataloader,model, criterion):
    acc = []
    loss_list = []
    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            logits, probs, _ = model(batch)
            loss = criterion(logits, batch.y)

            ## record
            _, prediction = torch.max(logits, -1)
            loss_list.append(loss.item())
            acc.append(prediction.eq(batch.y).cpu().numpy())

        eval_state = {'loss': np.average(loss_list),
                      'acc': np.concatenate(acc, axis=0).mean()}

    return eval_state


def test_GC(test_dataloader,model, criterion):
    acc = []
    loss_list = []
    pred_probs = []
    predictions = []
    model.eval()
    with torch.no_grad():
        for batch in test_dataloader:
            logits, probs, _ = model(batch)
            loss = criterion(logits, batch.y)

            # record
            _, prediction = torch.max(logits, -1)
            loss_list.append(loss.item())
            acc.append(prediction.eq(batch.y).cpu().numpy())
            predictions.append(prediction)
            pred_probs.append(probs)

    test_state = {'loss': np.average(loss_list),
                  'acc': np.average(np.concatenate(acc, axis=0).mean())}

    pred_probs = torch.cat(pred_probs, dim=0).cpu().detach().numpy()
    predictions = torch.cat(predictions, dim=0).cpu().detach().numpy()
    return test_state, pred_probs, predictions



### save path for model

In [13]:


if not os.path.isdir('checkpoint'):
    os.mkdir('checkpoint')
if not os.path.isdir(os.path.join('checkpoint', data_args.dataset_name)):
    os.mkdir(os.path.join('checkpoint', f"{data_args.dataset_name}"))
ckpt_dir = f"./checkpoint/{data_args.dataset_name}/"


def save_best(ckpt_dir, epoch, state, model_name, eval_acc, is_best, is_pruned):
    print('saving....')
    #model.to_device()
    state_save = {
        'net':state,
        'epoch':epoch,
        'acc': eval_acc 
        }
    best_pth_name = f'{model_name}_best.pth'
    fine_tuned_pth_name = f'{model_name}_fine_tuned_best.pth'
  
    if is_pruned & is_best:
        ckpt_path = os.path.join(ckpt_dir, fine_tuned_pth_name) 
        torch.save(state_save, ckpt_path)
    
     
    if is_pruned== False & is_best:
        ckpt_path = os.path.join(ckpt_dir, best_pth_name)  
        torch.save(state_save, ckpt_path)
           
        
    model.to_device()

### Implementing Global Pruning the Model and Re-Evaluate the Accuracy.
 - First, we determine the candidate layers for pruning. 

In [23]:
### So here is all parameters candidates for pruning
'''
parameters_to_prune = [
    (model.model.gnn_layers[0].lin, 'weight'),
    (model.model.gnn_layers[1].lin, 'weight'),
    (model.model.gnn_layers[2].lin, 'weight'),
    (model.model.mlps[0], 'weight')
]
'''

"\nparameters_to_prune = [\n    (model.model.gnn_layers[0].lin, 'weight'),\n    (model.model.gnn_layers[1].lin, 'weight'),\n    (model.model.gnn_layers[2].lin, 'weight'),\n    (model.model.mlps[0], 'weight')\n]\n"

### Criteria for measurements

In [14]:
### Setting Sparsity
sparsity=0.1

# The number of epochs  
num_epochs=10

# The number of iterations
num_iterations=2

In [15]:
# This is a dictionary to save all measurements. Aftre measuring, we can compute mean and std of each item.
Eva_final=dict()

# The following are all list of criteria for measurements. 
# We collect all desired datas of each list across iterations. 
# Then, we compute average and std of each list.


#Base model
Base_model_accuracy=[]
T_base_model=[]
Num_parm_base_model=[]
Base_model_size=[]
Base_Energy_Consumption=[]
Base_Cpu_Usage=[]
Base_Memory_Usage=[]

#Pruned model
Pruned_model_accuracy=[]
T_pruned_model=[]
Num_parm_pruned_model=[]
Pruned_model_size=[]
Pruned_Energy_Consumption=[]
Pruned_Cpu_Usage=[]
Pruned_Memory_Usage=[]

#Pruned and finetune model
Pruned_finetune_model_accuracy=[]
T_pruned_finetune_model=[]
Num_parm_pruned_finetune_model=[]
Pruned_finetune_model_size=[]
Pruned_finetune_Energy_Consumption=[]
Pruned_finetune_Cpu_Usage=[]
Pruned_finetune_Memory_Usage=[]

#recording sparsities of layers
Spar_gnn_layers_0_lin_w=[]
Spar_gnn_layers_1_lin_w=[]
Spar_gnn_layers_2_lin_w=[]
Spar_mlps_0=[]
Global_spar=[]   


# Here is the dictionary to record the list of all measurements
Eva_measure={'base model accuracy':Base_model_accuracy,
            'time inference of base model':T_base_model,
            'number parmameters of base model':Num_parm_base_model,
            'base model size':Base_model_size,
            'energy consumption of base model':Base_Energy_Consumption,
            'cpu usage of base model':Base_Cpu_Usage,
            'memory usage of base model':Base_Memory_Usage,
            'pruned model accuracy': Pruned_model_accuracy,
            'time inference of pruned model':T_pruned_model,
            'number parmameters of pruned model':Num_parm_pruned_model,
            'pruned model size':Pruned_model_size,
            'energy consumption of pruned model':Pruned_Energy_Consumption,
            'cpu usage of pruned model':Pruned_Cpu_Usage,
            'memory usage of pruned model':Pruned_Memory_Usage,
            'pruned finetune model accuracy':Pruned_finetune_model_accuracy,
            'time inference of pruned finetune model':T_pruned_finetune_model,
            'number parmameters of pruned finetune model':Num_parm_pruned_finetune_model,
            'pruned finetune model size':Pruned_finetune_model_size,
            'energy consumption of pruned_finetune model':Pruned_finetune_Energy_Consumption,
            'cpu usage of pruned_finetune model':Pruned_finetune_Cpu_Usage,
            'memory usage of pruned_finetune model':Pruned_finetune_Memory_Usage,
            'Sparsity in gnn_layers[0].lin.weight':Spar_gnn_layers_0_lin_w,
            'Sparsity in gnn_layers[1].lin.weight':Spar_gnn_layers_1_lin_w,
            'Sparsity in gnn_layers[2].lin.weight':Spar_gnn_layers_2_lin_w,
            'Sparsity in gmlps[0].weight':Spar_mlps_0,
            'Global sparsity':Global_spar
             
            }



### Training, Pruning, Finetuning

In [16]:
for i in range(num_iterations):
        print(f'This is iteration {i}')   
        print(f'Training and evaluation before pruning ')
        print("Starting training...")

        model = GnnNets(input_dim, output_dim, model_args)
        model.to_device()
        criterion = nn.CrossEntropyLoss()
        optimizer = Adam(model.parameters(), lr=train_args.learning_rate, weight_decay=train_args.weight_decay)

        #Training base model
        Eva=dict()# It is a dictionary to arrange output of this iteration
        best_acc=0  
        is_pruned=False
        for epoch in range(num_epochs):  
            acc=[]
            loss_list = []
            model.train()
            for batch in dataloader['train']:
                acc=train(callbacks=None)        

            eval_state = evaluate_GC(dataloader['eval'], model, criterion)
            acc_eval=eval_state['acc']
            # report train msg
            if epoch % 20 == 0:   
                print(f"Eval Epoch: {epoch} | Loss: {eval_state['loss']:.3f} | Acc: {eval_state['acc']:.3f}")


            # only save the best model
            if eval_state['acc'] > best_acc:
                early_stop_count = 0
            else:
                early_stop_count += 1

            if early_stop_count > train_args.early_stopping:
                break
            is_best = (eval_state['acc'] > best_acc)
            if is_best:
                best_acc = eval_state['acc']
                early_stop_count = 0
                    
            if is_best or epoch % train_args.save_epoch == 0:
                 save_best(ckpt_dir, epoch, model.state_dict(), model_args.model_name, eval_state['acc'], is_best, is_pruned)   
                                      



        #### load the best model
        base_model_path = os.path.join(ckpt_dir, f'{model_args.model_name}_best.pth') 
        checkpoint = torch.load(base_model_path)
        model.load_state_dict(checkpoint['net'])
        recover_model = lambda: model.load_state_dict(checkpoint['net'])

        # Start monitoring CPU and memory usage, model size, number of parametes, time inference and  power consumption
        gc.collect()
        time.sleep(5)  # Add a 5-second delay to stabilize the initial state
        tracemalloc.start()  # Start tracking memory allocations
        snapshot_before = tracemalloc.take_snapshot()#take a snapshot of the current memory state before starting the measurement.

        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)

        test_state, _, _ = test_GC(dataloader['test'], model, criterion)
        base_model_accuracy= test_state['acc']

        base_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_base_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        base_total_memory_diff = sum([stat.size_diff for stat in top_stats])
        base_energy_consumption = power_usage * t_base_model
        base_model_size = os.path.getsize(base_model_path)
        num_parm_base_model=get_num_parameters(model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5) 
        
        print(f'*****Results of base model*********')

        print(f"base model has accuracy on test set={base_model_accuracy:.2f}%")
        print(f"base model has size={base_model_size:.2f} byte")
        print(f"The time inference of base model is ={t_base_model}") 
        print(f"The number of parametrs of base model is:{num_parm_base_model}") 

        print(f"Energy Consumption : {base_energy_consumption:.3f}")
        print(f"total memory usage of base model':{base_total_memory_diff} ")
        print(f"cpu usage of base model':{base_cpu_usage:.3f} %")


        #Update Eva dictionary
        Eva.update({'base model accuracy': base_model_accuracy,
                    'time inference of base model': t_base_model,
                    'number parmameters of base model': num_parm_base_model,
                    'size of base model': base_model_size, 
                    'energy consumption of base model':base_energy_consumption,
                    'total memory usage of base model':base_total_memory_diff,
                    'cpu usage of base model':base_cpu_usage
                   })

        gc.collect()
        time.sleep(5)  

        print('_________******************************_____________')
        print(f'Pruning the Model')

        recover_model()
        parameters_to_prune = [
            (model.model.gnn_layers[0].lin, 'weight'),
            (model.model.gnn_layers[1].lin, 'weight'),
            (model.model.gnn_layers[2].lin, 'weight'),
            (model.model.mlps[0], 'weight')
        ]  
        pth_name = f"pruned_model.pth"   
        ckpt_pruned_path = os.path.join(ckpt_dir, pth_name)  
        global_pruning_torch(model,parameters_to_prune, sparsity, ckpt_pruned_path)
    
        #Loading the pruned model from disk
        pruned_model=load_and_evaluate_pruned_model(model_args, ckpt_pruned_path)
        
        #Sparsities of layyer
        spar_dict=get_model_sparsity(pruned_model)
        #recording sparsities of layers
        Eva.update(spar_dict)
     
 
        print('****************Result of pruning ******************')
       
        gc.collect()
        time.sleep(5)  
        tracemalloc.start()  
        snapshot_before = tracemalloc.take_snapshot()
        
        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)

        test_state, _, _ = test_GC(dataloader['test'], pruned_model, criterion)
        pruned_model_accuracy= test_state['acc']

        pruned_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_pruned_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        pruned_total_memory_diff = sum([stat.size_diff for stat in top_stats])
        pruned_energy_consumption = power_usage * t_pruned_model
        pruned_model_size = os.path.getsize(ckpt_pruned_path)
        num_parm_pruned_model=get_num_parameters(pruned_model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5)  
        
        ###### Report of pruning 
        print(f"pruned model has accuracy on test set={pruned_model_accuracy:.2f}%")
        print(f"pruned model has size={pruned_model_size:.2f} byte")
        print(f"The time inference of pruned model is ={t_pruned_model}") 
        print(f"The number of parametrs of pruned model is:{num_parm_pruned_model}") 

        print(f"Energy Consumption : {pruned_energy_consumption:.3f}")
        print(f"total memory usage of pruned model':{pruned_total_memory_diff} ")
        print(f"cpu usage of pruned model':{pruned_cpu_usage:.3f} %")


        #Update Eva dictionary
        Eva.update({'pruned model accuracy': pruned_model_accuracy,
                    'time inference of pruned model': t_pruned_model,
                    'number parmameters of pruned model': num_parm_pruned_model,
                    'size of pruned model': pruned_model_size, 
                    'energy consumption of pruned model':pruned_energy_consumption,
                    'total memory usage of pruned model':pruned_total_memory_diff,
                    'cpu usage of pruned model':pruned_cpu_usage
                   })

        gc.collect()
        time.sleep(5)   

        print('________*******************************_____________')
        print(f'Finetuning Pruned Sparse Model')

        best_sparse_checkpoint = dict()
        best_sparse_acc = 0
        is_pruned=True
        # The path of fine-tuned model
        fine_tuned_pth_name=f'{model_args.model_name}_fine_tuned_best.pth'
        fine_tuned_model_path = os.path.join(ckpt_dir, fine_tuned_pth_name)

        for epoch in range(num_epochs):
            # At the end of each train iteration, we have to apply the pruning mask
            #    to keep the model sparse during the training
            acc=[]
            loss_list = []

            for batch in dataloader['train']:
                acc=train(callbacks=[lambda: global_pruning_torch(model,parameters_to_prune, sparsity, fine_tuned_model_path)])        

            eval_state = evaluate_GC(dataloader['eval'], model, criterion)
            acc_eval=eval_state['acc']

            # report train msg
            if epoch % 20 == 0:   
                print(f"Eval Epoch: {epoch} | Loss: {eval_state['loss']:.3f} | Acc: {eval_state['acc']:.3f}")


            # only save the best model
            if eval_state['acc'] > best_sparse_acc :
                early_stop_count = 0
            else:
                early_stop_count += 1

            if early_stop_count > train_args.early_stopping:
                break
            is_best = (eval_state['acc'] > best_sparse_acc )
           
            if is_best:
                best_acc = eval_state['acc']
                early_stop_count = 0
               
        
            #if is_best or epoch % train_args.save_epoch == 0:
                #Remove Zeroed Weights 
                #sparse_zero_state=state_sparse_model(model, eval_state['acc'])
                #save_best(ckpt_dir, epoch,sparse_zero_state, model_args.model_name, eval_state['acc'], is_best, is_pruned)

                
        # load the best fine-tune model
        sparse_model =load_and_evaluate_pruned_model(model_args, fine_tuned_model_path)

        # Now you can use the sparse model for evaluation or further training
        print('****************Result of fine-tuning of pruned model ******************')

        gc.collect()
        time.sleep(5)  
        tracemalloc.start() 
        snapshot_before = tracemalloc.take_snapshot()

        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)

        test_state, _, _ = test_GC(dataloader['test'], sparse_model, criterion)
        pruned_finetune_model_accuracy= test_state['acc']

        pruned_finetune_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_pruned_finetune_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        pruned_finetune_total_memory_diff = sum([stat.size_diff for stat in top_stats])
        pruned_finetune_energy_consumption = power_usage * t_pruned_finetune_model
        pruned_finetune_model_size = os.path.getsize( fine_tuned_model_path)
        num_parm_pruned_finetune_model=get_num_parameters(sparse_model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5)  # Add a 5-second delay to stabilize the initial state    
        
        ###### Report  
    
        print(f"pruned_finetune model has accuracy on test set={pruned_finetune_model_accuracy:.2f}%")
        print(f"pruned_finetune model has size={pruned_finetune_model_size:.2f} byte")
        print(f"The time inference of pruned_finetune model is ={t_pruned_finetune_model}") 
        print(f"The number of parametrs of pruned_finetune model is:{num_parm_pruned_finetune_model}") 

        print(f"Energy Consumption of pruned_finetune model: {pruned_finetune_energy_consumption:.3f}")
        print(f"total memory usage of pruned_finetune model':{pruned_finetune_total_memory_diff} ")
        print(f"cpu usage of pruned_finetune model':{pruned_finetune_cpu_usage:.3f} %")


        #Update my Eva dictionary
        Eva.update({'pruned and finetune model accuracy': pruned_finetune_model_accuracy,
                    'time inference of pruned and finetune model': t_pruned_finetune_model,
                    'number parmameters of pruned and finetune model': num_parm_pruned_finetune_model,
                    'size of pruned and finetune model': pruned_finetune_model_size, 
                    'energy consumption of pruned and finetune model':pruned_finetune_energy_consumption,
                    'total memory usage of pruned and finetune model':pruned_finetune_total_memory_diff,
                    'cpu usage of pruned and finetune model':pruned_finetune_cpu_usage
                   })

        gc.collect()
        time.sleep(5) 


        Base_model_accuracy.append(Eva['base model accuracy'])
        T_base_model.append(Eva['time inference of base model'])
        Num_parm_base_model.append(int(Eva['number parmameters of base model']))
        Base_model_size.append(int(Eva['size of base model']))
        Base_Energy_Consumption.append(Eva['energy consumption of base model'])
        Base_Cpu_Usage.append(Eva['cpu usage of base model'])
        Base_Memory_Usage.append(Eva['total memory usage of base model'])

        Pruned_model_accuracy.append(Eva['pruned model accuracy'])
        T_pruned_model.append(Eva['time inference of pruned model'])
        Num_parm_pruned_model.append(int(Eva['number parmameters of pruned model']))
        Pruned_model_size.append(int(Eva['size of pruned model']))
        Pruned_Energy_Consumption.append(Eva['energy consumption of pruned model'])
        Pruned_Cpu_Usage.append(Eva['cpu usage of pruned model'])
        Pruned_Memory_Usage.append(Eva['total memory usage of pruned model'])


        Pruned_finetune_model_accuracy.append(Eva['pruned and finetune model accuracy'])
        T_pruned_finetune_model.append(Eva['time inference of pruned and finetune model'])
        Num_parm_pruned_finetune_model.append(int(Eva['number parmameters of pruned and finetune model']))
        Pruned_finetune_model_size.append(int(Eva['size of pruned and finetune model']))
        Pruned_finetune_Energy_Consumption.append(Eva['energy consumption of pruned and finetune model'])
        Pruned_finetune_Cpu_Usage.append(Eva['cpu usage of pruned and finetune model'])
        Pruned_finetune_Memory_Usage.append(Eva['total memory usage of pruned and finetune model'])


        Spar_gnn_layers_0_lin_w.append(Eva['model.gnn_layers.0.lin.weight'])
        Spar_gnn_layers_1_lin_w.append(Eva['model.gnn_layers.1.lin.weight'])
        Spar_gnn_layers_2_lin_w.append(Eva['model.gnn_layers.2.lin.weight'])
        Spar_mlps_0.append(Eva['model.mlps.0.weight'])
        Global_spar.append(Eva['Global sparsity'])    



This is iteration 0
Training and evaluation before pruning 
Starting training...
Eval Epoch: 0 | Loss: 0.536 | Acc: 0.754
saving....
saving....
saving....
saving....
*****Results of base model*********
base model has accuracy on test set=0.76%
base model has size=141721.00 byte
The time inference of base model is =2.2245275999885052
The number of parametrs of base model is:34520
Energy Consumption : 76.190
total memory usage of base model':33281 
cpu usage of base model':55.800 %
_________******************************_____________
Pruning the Model
Pruned model loaded.
Sparsity in model.gnn_layers.0.lin.weight: 0.058
Sparsity in model.gnn_layers.1.lin.weight: 0.099
Sparsity in model.gnn_layers.2.lin.weight: 0.102
Sparsity in model.mlps.0.weight: 0.211
Global sparsity: 0.100
****************Result of pruning ******************
pruned model has accuracy on test set=0.28%
pruned model has size=127105.00 byte
The time inference of pruned model is =2.2747928999888245
The number of parametr

### Computing the mean and std 

In [117]:
Eva_final=dict()
base_model_accuracy_mean = stat.mean(Base_model_accuracy)
base_model_accuracy_std =  stat.stdev(Base_model_accuracy)
#desc = "{:.3f} ± {:.3f}".format(base_model_accuracy_mean,base_model_accuracy_std)


Eva_final.update({'Ave of base model accuracy':float(format(base_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of base model accuracy':float(format(base_model_accuracy_std, '.3f'))})
                 
t_base_model_mean =stat.mean(T_base_model)
t_base_model_std =stat.stdev(T_base_model)  
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of time inference of base model':float(format(t_base_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of base model':float(format(t_base_model_std, '.3f'))})


num_parm_base_model_mean = stat.mean(Num_parm_base_model)
num_parm_base_model_std = stat.stdev(Num_parm_base_model)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of number parmameters of base model':num_parm_base_model_mean})
Eva_final.update({'Std of number parmameters of base model':num_parm_base_model_std})

base_model_size_mean = stat.mean(Base_model_size)
base_model_size_std = stat.stdev(Base_model_size)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of base model size':base_model_size_mean})
Eva_final.update({'Std of base model size':base_model_size_std})


base_energy_consumption_mean = stat.mean(Base_Energy_Consumption)
base_energy_consumption_std = stat.stdev(Base_Energy_Consumption)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of energy consumption of base model':base_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of base model':base_energy_consumption_std})


base_cpu_usage_mean = stat.mean(Base_Cpu_Usage)
base_cpu_usage_std = stat.stdev(Base_Cpu_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of cpu usage of base model':base_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of base model':base_cpu_usage_std})

base_memory_usage_mean = stat.mean(Base_Memory_Usage)
base_memory_usage_std = stat.stdev(Base_Memory_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of memory usage of base model':base_memory_usage_mean})
Eva_final.update({'Std of memory usage of base model':base_memory_usage_std})




#################################

pruned_model_accuracy_mean =stat.mean(Pruned_model_accuracy)
pruned_model_accuracy_std = stat.stdev(Pruned_model_accuracy)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of pruned model accuracy':float(format(pruned_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of pruned model accuracy':float(format(pruned_model_accuracy_std, '.3f'))})
                 

t_pruned_model_mean = stat.mean(T_pruned_model)
t_pruned_model_std =stat.stdev(T_pruned_model)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of time inference of pruned model':float(format(t_pruned_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of pruned model':float(format(t_pruned_model_std, '.3f'))})

num_parm_pruned_model_mean = stat.mean(Num_parm_pruned_model)
num_parm_pruned_model_std = stat.stdev(Num_parm_pruned_model)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of number parmameters of pruned model':num_parm_pruned_model_mean})
Eva_final.update({'Std of number parmameters of pruned model':num_parm_pruned_model_std})

pruned_model_size_mean =stat.mean( Pruned_model_size)
pruned_model_size_std = stat.stdev(Pruned_model_size)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of pruned model size':pruned_model_size_mean})
Eva_final.update({'Std of pruned model size':pruned_model_size_std })

pruned_energy_consumption_mean = stat.mean(Pruned_Energy_Consumption)
pruned_energy_consumption_std = stat.stdev(Pruned_Energy_Consumption)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of energy consumption of pruned model':pruned_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of pruned model':pruned_energy_consumption_std})


pruned_cpu_usage_mean = stat.mean(Pruned_Cpu_Usage)
pruned_cpu_usage_std = stat.stdev(Pruned_Cpu_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of cpu usage of pruned model':pruned_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of pruned model':pruned_cpu_usage_std})

pruned_memory_usage_mean = stat.mean(Pruned_Memory_Usage)
pruned_memory_usage_std = stat.stdev(Pruned_Memory_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of memory usage of pruned model':pruned_memory_usage_mean})
Eva_final.update({'Std of memory usage of pruned model':pruned_memory_usage_std})


#################################
pruned_finetune_model_accuracy_mean =stat.mean(Pruned_finetune_model_accuracy)
pruned_finetune_model_accuracy_std = stat.stdev(Pruned_finetune_model_accuracy)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of pruned finetune model accuracy':float(format(pruned_finetune_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of pruned finetune model accuracy':float(format(pruned_finetune_model_accuracy_std, '.3f'))})                 

t_pruned_finetune_model_mean =stat.mean(T_pruned_finetune_model)
t_pruned_finetune_model_std =stat.stdev(T_pruned_finetune_model)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of time inference of pruned finetune model':float(format(t_pruned_finetune_model_mean,'.3f'))})
Eva_final.update({'Std of time inference of pruned finetune model':float(format(t_pruned_finetune_model_std,'.3f'))})

num_parm_pruned_finetune_model_mean =stat.mean(Num_parm_pruned_finetune_model)
num_parm_pruned_finetune_model_std = stat.stdev(Num_parm_pruned_finetune_model)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of number parmameters of pruned finetune model':num_parm_pruned_finetune_model_mean})
Eva_final.update({'Std of number parmameters of pruned finetune model':num_parm_pruned_finetune_model_std })

pruned_finetune_model_size_mean = stat.mean(Pruned_finetune_model_size)
pruned_finetune_model_size_std = stat.stdev(Pruned_finetune_model_size)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of pruned finetune model size':pruned_finetune_model_size_mean})
Eva_final.update({'Std of pruned finetune model size':pruned_finetune_model_size_std})


pruned_finetune_energy_consumption_mean = stat.mean(Pruned_finetune_Energy_Consumption)
pruned_finetune_energy_consumption_std = stat.stdev(Pruned_finetune_Energy_Consumption)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of energy consumption of pruned_finetune model':pruned_finetune_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of pruned_finetune model':pruned_finetune_energy_consumption_std})


pruned_finetune_cpu_usage_mean = stat.mean(Pruned_finetune_Cpu_Usage)
pruned_finetune_cpu_usage_std = stat.stdev(Pruned_finetune_Cpu_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of cpu usage of pruned_finetune model':pruned_finetune_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of pruned_finetune model':pruned_finetune_cpu_usage_std})

pruned_finetune_memory_usage_mean = stat.mean(Pruned_finetune_Memory_Usage)
pruned_finetune_memory_usage_std = stat.stdev(Pruned_finetune_Memory_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of memory usage of pruned_finetune model':pruned_finetune_memory_usage_mean})
Eva_final.update({'Std of memory usage of pruned_finetune model':pruned_finetune_memory_usage_std})



sparsity_gnn_layers_0_lin_w_mean = stat.mean(Spar_gnn_layers_0_lin_w)
sparsity_gnn_layers_0_lin_w_std = stat.stdev(Spar_gnn_layers_0_lin_w)
Eva_final.update({'Sparsity in gnn_layers[0].lin.weight':float(format(sparsity_gnn_layers_0_lin_w_mean,'.3f'))})
Eva_final.update({'Std of Sparsity in gnn_layers[0].lin.weight':float(format(sparsity_gnn_layers_0_lin_w_std,'.3f'))})

sparsity_gnn_layers_1_lin_w_mean = stat.mean(Spar_gnn_layers_1_lin_w)
sparsity_gnn_layers_1_lin_w_std = stat.stdev(Spar_gnn_layers_1_lin_w)
Eva_final.update({'Sparsity in gnn_layers[1].lin.weight':float(format(sparsity_gnn_layers_1_lin_w_mean,'.3f'))})
Eva_final.update({'Std of Sparsity in gnn_layers[1].lin.weight':float(format(sparsity_gnn_layers_1_lin_w_std,'.3f'))})


sparsity_gnn_layers_2_lin_w_mean = stat.mean(Spar_gnn_layers_2_lin_w)
sparsity_gnn_layers_2_lin_w_std = stat.stdev(Spar_gnn_layers_2_lin_w)
Eva_final.update({'Sparsity in gnn_layers[2].lin.weight':float(format(sparsity_gnn_layers_2_lin_w_mean,'.3f'))}) 
Eva_final.update({'Std of Sparsity in gnn_layers[2].lin.weight':float(format(sparsity_gnn_layers_2_lin_w_std,'.3f'))}) 

sparsity_mlps_0_mean = stat.mean(Spar_mlps_0)
sparsity_mlps_0_std = stat.stdev(Spar_mlps_0)
Eva_final.update({'Sparsity in gmlps[0].weight':float(format(sparsity_mlps_0_mean,'.3f')) })
Eva_final.update({'Std of Sparsity in gmlps[0].weight':float(format(sparsity_mlps_0_std,'.3f')) })


Global_sparsity_mean = stat.mean(Global_spar)
Global_sparsity_std = stat.stdev(Global_spar)
Eva_final.update({'Global sparsity':float(format(Global_sparsity_mean,'.3f'))})
Eva_final.update({'Std of Global sparsity':float(format(Global_sparsity_std,'.3f'))})


#################################


print(f"All measurement about pruning process of sparsity:{sparsity*100}% ")   
Eva_final

All measurement about pruning process of sparsity:90.0% 


{'Ave of base model accuracy': 0.768,
 'Std of base model accuracy': 0.017,
 'Ave of time inference of base model': 2.156,
 'Std of time inference of base model': 0.025,
 'Ave of number parmameters of base model': 34513.5,
 'Std of number parmameters of base model': 4.949747468305833,
 'Ave of base model size': 141721,
 'Std of base model size': 0.0,
 'Ave of energy consumption of base model': 49.855076172416375,
 'Std of energy consumption of base model': 19.70535232167039,
 'Ave of cpu usage of base model': 27.3,
 'Std of cpu usage of base model': 31.53696244092002,
 'Ave of memory usage of base model': 27853.5,
 'Std of memory usage of base model': 2013.1330060381008,
 'Ave of pruned model accuracy': 0.717,
 'Std of pruned model accuracy': 0.0,
 'Ave of time inference of pruned model': 2.132,
 'Std of time inference of pruned model': 0.01,
 'Ave of number parmameters of pruned model': 3755.5,
 'Std of number parmameters of pruned model': 4.949747468305833,
 'Ave of pruned model size

In [118]:
## Recording result
### The sparsity changes across range(0, 1, .01)
dataset_name = 'BBBP'
Pruning_Method='Global_Pruning'
max_epoch = 100
resume = True
result_folder ='pathresult/'
if not os.path.exists(result_folder):
    os.makedirs(result_folder)



file_name = result_folder+Pruning_Method+'_'+'with sparsity of'+'_'+str(sparsity)+'_on_'+dataset_name+'_'+str(max_epoch)+'.txt'

with open(file_name, 'w') as f:
    f.write('%s:%s\n'%('dataset_name', 'BBBP'))
    f.write('%s:%s\n'%('max_epoch', max_epoch))
    f.write('%s:%s\n'%('sparsity', sparsity))
    
    for key, value in Eva_final.items():
        f.write('%s:%s\n'%(key, value))
        
    for key, value in Eva_measure.items():
        f.write('%s:%s\n' % (key, ','.join(map(str, value))))            
                   
       